In [69]:
import pandas as pd 
import numpy as np

In [70]:
edx=pd.read_csv('C:/Users/nehab/Desktop/Python/Webscraping/Edx_Courses_Data_New_Final11.csv')

In [71]:
edx.shape

(1189, 15)

In [72]:
edx.head()

,Course Name,Multi Categories,Course URL,Course Provider,Short Description,Long Description,Difficulty,Duration,Commitment,Number of Enrolled,Cost,Alternative Options,Language,Video Language,Prerequisites
0,4G Network Essentials,Engineering,https://www.edx.org/course/4g-network-essentials,NaN,Advance your telecom career by learning how 4G...,4G is the cutting-edge network technology that...,Intermediate,6 weeks,3–4 hours per week,NaN,Free,"₹ 4,007",English,English,"BSc degree (algebra, basic mathematics concep..."
1,5G Deployment,"Computer Science, Engineering",https://www.edx.org/course/5g-deployment,NaN,"Learn about 5G deployment, 5G Radio Access Net...","In this course, you will learn about 5G deploy...",Introductory,6 weeks,4–6 hours per week,NaN,Free,"₹ 16,273",English,English,There are no pre-requisites for this course.
2,5G Essentials,"Computer Science, Engineering",https://www.edx.org/course/5g-essentials,Curtin University,"Learn about 5G Networking, 5G architecture, an...","In this course, you will learn and explore abo...",Introductory,7 weeks,4–6 hours per week,NaN,Free,"₹ 16,273",English,English,There are no prerequisites for this course.
3,AI Applications with Watson,Computer Science,https://www.edx.org/course/AI-applications-wit...,NaN,Turbocharge your chatbot's IQ and AI capabilit...,Please Note: Learners who successfully complet...,Intermediate,3 weeks,2–4 hours per week,"13,695",Free,"₹ 8,096",English,English,basic knowledge of object-oriented programming...
4,AI Chatbots without Programming,Computer Science,https://www.edx.org/course/AI-chatbots-without...,NaN,Chatbots are increasingly in demand among glob...,Please Note: Learners who successfully complet...,Introductory,2 weeks,2–4 hours per week,"48,488",Free,"₹ 8,096",English,English,None


### Preprocess difficulty column 

In [73]:
print('All levels of difficulty:')
print(edx['Difficulty'].unique())

All levels of difficulty:
['Intermediate' 'Introductory' 'Advanced' nan]


In [74]:
edx['Difficulty'].isnull().sum()

1

In [75]:
edx.dropna(subset=['Difficulty'],inplace=True)

In [76]:
edx['Difficulty'].isnull().sum()


0

In [77]:

# Convert course difficulty from string to numbers in increasing difficulty order

# 0: introductory , 1:Intermediate , 2: Advanced 
def conv_diff(diff_string):

  if diff_string == 'Introductory':
    diff = 0
  elif diff_string == 'Intermediate':
    diff = 1
  else:
    diff = 2
  return diff

In [78]:
# Extract Course Difficulty, one-hot encoded

print('All levels of difficulty:')
print(edx['Difficulty'].unique())

data_difficulty = edx['Difficulty'].apply(lambda x: conv_diff(x))

All levels of difficulty:
['Intermediate' 'Introductory' 'Advanced']


# Preprocess Duration 

In [79]:
edx['Duration'].isnull().sum()
edx.dropna(subset=['Duration'],inplace=True)
print(edx['Duration'].isnull().sum())

0


In [80]:
edx['Commitment'].isnull().sum()
edx.dropna(subset=['Commitment'],inplace=True)
print(edx['Commitment'].isnull().sum())

0


In [81]:
# Get course week duration given a string of 'Course Duration'

def cal_cw(course_duration):
  if pd.isna(course_duration):
    cw = np.nan
  else:
    cw = course_duration.split(' ')[0]
    cw = int(cw)
  return cw


In [82]:
# Get average hours per week given a string of 'Commitment'

def cal_ahpw(commitment):
  if pd.isna(commitment):
    ahpw = np.nan
  else:
    list_hours = commitment.split(' ')[0].split('–')
    list_hours = [int(x) for x in list_hours]
    if len(list_hours) == 1:
      ahpw = list_hours[0]
    else:
      ahpw = np.mean(list_hours)
  return ahpw
  

In [83]:
# Convert course duration in hours to categorical
# 0 : short , 1 : medium , 2 : long
def conv_dur(dur):
  if pd.isna(dur):
    cat_dur = np.nan
  elif (dur > 0) and (dur <= 10):
    cat_dur = 0
  elif (dur > 10) and (dur <= 50):
    cat_dur = 1
  else:
    cat_dur = 2
  return cat_dur

In [84]:
# Filter for non empty value Course Duration
week_non = np.array((edx['Duration'].isna() == False))


In [85]:
# Filter for non empty value Commitment 
com_non= np.array((edx['Commitment'].isna() == False))


In [86]:
# Filter for non empty value for both Course Duration and Commitment
dur_non = week_non * com_non
dur_non

array([ True,  True,  True, ...,  True,  True,  True])

In [87]:
dur_non.sum()

1183

In [88]:
# Calculate global average course duration

avg_dur = (np.array(edx['Duration'][dur_non].apply(lambda x: cal_cw(x))) * np.array(edx['Commitment'][dur_non].apply(lambda x: cal_ahpw(x)))).sum()  / dur_non.sum()
avg_dur

36.30050718512257

In [89]:
# Calculate course duration in hours
# cw - course week duration
# ahpw - average hours per week
# ahpw = cw * ahpw

def cal_dur(week_dur, commt):
  cw = cal_cw(week_dur)
  ahpw = cal_ahpw(commt)
  if pd.isna(cw):
    dur = np.nan
  elif (not(pd.isna(cw)) and pd.isna(ahpw)):
    dur = avg_dur # for courses with number of weeks but not average hours per week info
  else: 
    dur = cw*ahpw
  return conv_dur(dur)

In [90]:
# Get Course Duration in unit of hours
dur_hr = edx[['Duration',	'Commitment']].apply(lambda x: cal_dur(x[0],x[1]), axis=1)

###  Get Course Name, url , category 

In [91]:
data_name = edx['Course Name']
data_url = edx['Course URL']
data_cat = edx['Multi Categories']

### Get course description

In [92]:
# Extract Course Short Description
data_sdesc = edx['Short Description'].apply(lambda x: '' if pd.isna(x) else x)

In [93]:
# Extract Course Long Description

data_ldesc = edx['Long Description'].apply(lambda x: '' if pd.isna(x) else x)

### get cost column



In [94]:
# Check if free option is available
# 1 : Free , 0: not free
data_costcat =edx['Cost'].apply(lambda x: 1 if x == 'Free' else 0)

In [100]:
# Get Cost for paid option
data_paid = edx['Alternative Options']

### Rating 

In [96]:

data_enroll = edx['Number of Enrolled'].apply(lambda x: '0' if pd.isna(x) else x)
data_enroll = data_enroll.apply(lambda x: x.replace(',', ''))
data_enroll = data_enroll.apply(lambda x: int(x))

In [98]:
data_rating = np.zeros(data_enroll.shape[0])
data_rating

array([0., 0., 0., ..., 0., 0., 0.])

In [102]:
# Get course list of subtitle language available

data_sublang = edx['Video Language']

In [103]:
# Add Platform name
# 0 - Edx

data_plat = np.zeros(edx.shape[0])

In [105]:
# Course Provider

data_prov = edx['Course Provider']

In [115]:
# get courde img url
edx['Course Img']='https://www.edx.org/images/logos/edx-logo-elm.svg'
data_img = edx['Course Img']

## Combine data
 